### Laboratorio 6 - Generative Adversarial Network
Stefano Aragoni, Carol Arévalo

-----------

En esta práctica se diseñó una Generative Adversarial Network (GAN) con el propósito de poder generar imágenes artificiales que imiten la distribución de los datos originales Para esto, fue necesario diseñar una red neuronal que fuera capaz de generar imágenes, y otra red neuronal que fuera capaz de diferenciar entre imágenes reales y generadas. 

A continuación se muestra el código utilizado para la creación de la GAN, así como los resultados obtenidos.

------- 

##### Importar librerías

Como primer paso, se importaron las librerías necesarias para la creación de la GAN.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input
from keras.optimizers import Adam
import os
import cv2


--------
### **Preparación de Datos**

##### Cargar el dataset de CelebA

Posteriormente, se descargó el dataset de CelebA. Este conjunto de datos consta de más de 200,000 imágenes a color, de 128 X 128 X 3 c/u.

In [1]:
fotos_dir = 'archive/img_align_celeba'
bounding_boxes_file = 'archive/list_bbox_celeba.csv'


# Cargar las imágenes a un array de numpy. 
for foto in os.listdir(fotos_dir):
    # Leer la imagen
    # Añadir la imagen al array de numpy
    # Si el array de numpy ya tiene 1000 imágenes, parar el bucle
    pass

##### Preprocesamiento: Normalización, recorte y redimensionamiento de las imágenes.

Con los datos cargados, se prosiguió a normalizar, recortar, y redimensionar las imágenes.

--------
### **Implementación de la GAN**

##### Diseño del generador y el discriminador

A continuación se demuestra el modelo del <font color=orange>generador</font>.

A continuación se demuestra el modelo del <font color=orange>discriminador</font>.

##### Definición de funciones de pérdida y optimizadores

Con los modelos listos, se procedió a definir las funciones de pérdida y los optimizadores. Esto con el propósito de poder entrenar la GAN. Más específicamente, se utilizó la función de <font color=orange>Binary Cross Entropy</font> (BCE) como función de pérdida, y el <font color=orange>optimizador Adam</font>. 

Asimismo, se indicó que el discriminador no se entrenaría durante el entrenamiento de la GAN, ya que el objetivo es entrenar al generador para que engañe al discriminador.

--------
### **Entrenamiento de la GAN**

##### Implementación del bucle de entrenamiento

##### Visualización de los resultados durante el entrenamiento

--------
### **Reflexión**

Reflexione sobre lo aprendido en la sesión teórica y cómo se aplicó en el laboratorio. Algunos 
puntos que podrían considerar en su reflexión incluyen:

1. ¿Qué conceptos de la teoría encontraron más desafiantes y por qué?


2. ¿Cómo les ayudó el laboratorio a consolidar o entender mejor estos conceptos?


3. ¿Qué aplicaciones potenciales ven para las GANs en la industria o en la investigación?

4. ¿Qué limitaciones o preocupaciones éticas pueden identificar en el uso de GANs?


5. ¿Cómo se sienten con respecto a la implementación y entrenamiento de GANs después de la  experiencia práctica?